Set Up
=================

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from tensorflow.keras import layers
from functools import partial
import copy
import os
import pandas as pd


PATH = '/content/MedMNIST/MedMNIST'

###Training part starts here. For test answer generation, go to **Load Model and Test Image** part.


---



Unzip All File
=================

In [ ]:
! unzip -o /content/ucsc-cse-164-spring-2022-final-project.zip -d /content/MedMNIST

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: /content/MedMNIST/MedMNIST/unlabeled/wd7e02foa4lx5bmn3yjg.jpeg  
  inflating: /content/MedMNIST/MedMNIST/unlabeled/wd8etjl53gbi7cr0u219.jpeg  
  inflating: /content/MedMNIST/MedMNIST/unlabeled/wd8jr70uiefpyoal326t.jpeg  
  inflating: /content/MedMNIST/MedMNIST/unlabeled/wd8xsfvrnjel5g197thp.jpeg  
  inflating: /content/MedMNIST/MedMNIST/unlabeled/wd912m7eo86pui5nqyxf.jpeg  
  inflating: /content/MedMNIST/MedMNIST/unlabeled/wd9r8fb426hzcigl3kyu.jpeg  
  inflating: /content/MedMNIST/MedMNIST/unlabeled/wdba49eg1v7j6l8ynckt.jpeg  
  inflating: /content/MedMNIST/MedMNIST/unlabeled/wdcmep8szni6q1tkbfal.jpeg  
  inflating: /content/MedMNIST/MedMNIST/unlabeled/wdhu1kcay0v4ge9on7fj.jpeg  
  inflating: /content/MedMNIST/MedMNIST/unlabeled/wdibvl14hyca2qs6ujzx.jpeg  
  inflating: /content/MedMNIST/MedMNIST/unlabeled/wdje573ftlkc0bi4pq1u.jpeg  
  inflating: /content/MedMNIST/MedMNIST/unlabeled/wdjgzibhkyx4f8r7cl2t.jpeg  
  inflating: /content/MedMNIST/MedMNI

Fix Seed for Reproducible Results
==================

In [ ]:
seed = 1
np.random.seed(seed)
tf.random.set_seed(seed)

Load Data
=================

In [ ]:
# file information
MAX_UNLABEL_NUM = 49834 # all:49834

# load labeled data
labeled_train_dataset = []
for i, line in enumerate(open(PATH+"/wnids.txt", "r")):
  class_name = line.rstrip("\n")
  labeled_train_dataset.append(tf.data.Dataset.list_files(PATH+'/lableed/'+class_name+'/*.jpeg'))

labels = [0, 1, 2, 3, 4, 5]
labeled_train_images, labeled_train_labels = [], []
def read_labeled_image(path_list, label):
    for i in path_list:
        image_temp = tf.io.read_file(i)
        image_temp = tf.image.decode_jpeg(image_temp)
        labeled_train_images.append(image_temp)
        labeled_train_labels.append(label)
    return 0
for label in labels:
  read_labeled_image(path_list=labeled_train_dataset[label],label=label)
labeled_train_images = np.array(labeled_train_images)
labeled_train_labels = np.array(labeled_train_labels)
print("labeled data:", labeled_train_images.shape)

# load unlabeled data
unlabeled_train_images = []
unlabeled_train_dataset = tf.data.Dataset.list_files(PATH+'/unlabeled/*.jpeg')
for i in unlabeled_train_dataset:
    image_temp = tf.io.read_file(i)
    image_temp = tf.image.decode_jpeg(image_temp)
    unlabeled_train_images.append(image_temp)
    if len(unlabeled_train_images) == MAX_UNLABEL_NUM:
      break
unlabeled_train_images = np.array(unlabeled_train_images)
print("unlabeled data:", unlabeled_train_images.shape)

# Model / data parameters
num_classes = 6
input_shape = (64, 64, 1)

# Scale images to the [0, 1] range
labeled_x_train = labeled_train_images.astype("float32") / 255
unlabeled_x_train = unlabeled_train_images.astype("float32") / 255
# Make sure images have shape (64, 64, 1)
labeled_x_train = np.expand_dims(labeled_x_train, -1)
unlabeled_x_train = np.expand_dims(unlabeled_x_train, -1)

# convert class vectors to binary class matrices
labeled_y_train = keras.utils.to_categorical(labeled_train_labels, num_classes)
unlabeled_y_train = np.zeros((MAX_UNLABEL_NUM, 1))
print(labeled_y_train.shape, unlabeled_y_train.shape)


labeled data: (60, 64, 64, 1)
unlabeled data: (49834, 64, 64, 1)
(60, 6) (49834, 1)


Create Dataset
==================

In [ ]:
BATCH_SIZE = 10
SHUFFLE_BUFFER_SIZE = 500

labeled_train_dataset = tf.data.Dataset.from_tensor_slices((labeled_x_train, labeled_y_train))
labeled_train_dataset = labeled_train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
unlabeled_train_dataset = tf.data.Dataset.from_tensor_slices((unlabeled_x_train, unlabeled_y_train))
unlabeled_train_dataset = unlabeled_train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
train_dataset = tf.data.Dataset.zip((unlabeled_train_dataset, labeled_train_dataset)) #.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


Semisupervised Model
=================

In [ ]:
class SemisupervisedModel(keras.Model):
    def __init__(self, model, entropy=False, lamda=0.1):
        super().__init__()

        self.model = model
        self.cce = tf.keras.losses.CategoricalCrossentropy()

        self.entropy = entropy
        self.lamda = lamda

    def compile(self, optimizer, **kwargs):
        super().compile(**kwargs)

        self.optimizer = optimizer
        self.accuracy_tracker = keras.metrics.Accuracy()
        self.loss_tracker = keras.metrics.Mean(name="loss")

    @property
    def metrics(self):
        return [self.loss_tracker, self.accuracy_tracker]
        # return [self.loss_tracker]

    def call(self, images, training=True):
        return self.model(images, training=training)

    def semisupervised_loss(self, labeled_y_true, labeled_y_pred, unlabeled_y_true, unlabeled_y_pred):
        loss = self.cce(labeled_y_true, labeled_y_pred)
        if self.entropy:
            loss += -self.lamda * tf.reduce_mean(tf.reduce_sum(unlabeled_y_pred * tf.math.log(unlabeled_y_pred + 1e-12), axis=1))
        return loss

    def train_step(self, data):
        (unlabeled_images, unlabeled_y_true), (labeled_images, labeled_y_true) = data

        with tf.GradientTape() as tape:
            labeled_y_pred = self.call(labeled_images, training=True)
            unlabeled_y_pred = self.call(unlabeled_images, training=True)
            loss = self.semisupervised_loss(labeled_y_true, labeled_y_pred, unlabeled_y_true, unlabeled_y_pred)
        gradients = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))
        self.loss_tracker.update_state(loss)

        self.accuracy_tracker.update_state(tf.math.argmax(unlabeled_y_true, axis=1), tf.math.argmax(unlabeled_y_pred, axis=1))
        self.accuracy_tracker.update_state(tf.math.argmax(labeled_y_true, axis=1), tf.math.argmax(labeled_y_pred, axis=1))

        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        images, labels = data
        y_pred = self.call(images, training=False)

        loss = self.cce(labels, y_pred)
        self.loss_tracker.update_state(loss)

        self.accuracy_tracker.update_state(tf.math.argmax(labels, axis=1), tf.math.argmax(y_pred, axis=1))

        return {m.name: m.result() for m in self.metrics}

~~Naive Baseline~~
=================

In [ ]:
'''model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)
model.summary()
'''

'model = keras.Sequential(\n    [\n        keras.Input(shape=input_shape),\n        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),\n        layers.MaxPooling2D(pool_size=(2, 2)),\n        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),\n        layers.MaxPooling2D(pool_size=(2, 2)),\n        layers.Flatten(),\n        layers.Dropout(0.5),\n        layers.Dense(num_classes, activation="softmax"),\n    ]\n)\nmodel.summary()\n'

In [ ]:
'''semi_model = SemisupervisedModel(
    keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
    ),
    entropy=True
)'''

'semi_model = SemisupervisedModel(\n    keras.Sequential(\n    [\n        keras.Input(shape=input_shape),\n        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),\n        layers.MaxPooling2D(pool_size=(2, 2)),\n        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),\n        layers.MaxPooling2D(pool_size=(2, 2)),\n        layers.Flatten(),\n        layers.Dropout(0.5),\n        layers.Dense(num_classes, activation="softmax"),\n    ]\n    ),\n    entropy=True\n)'

In [ ]:
'''epochs = 20

semi_model.compile(optimizer=keras.optimizers.Adam(), metrics=["accuracy"])

semi_model.fit(train_dataset, epochs=epochs)'''

'epochs = 20\n\nsemi_model.compile(optimizer=keras.optimizers.Adam(), metrics=["accuracy"])\n\nsemi_model.fit(train_dataset, epochs=epochs)'

Deeper CNN
=================

In [ ]:
semi_model_1 = SemisupervisedModel(
    keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), padding="same", activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), padding="same", activation="relu"),
        layers.Conv2D(64, kernel_size=(3, 3), padding="same", activation="relu"),
        layers.Conv2D(64, kernel_size=(3, 3), padding="same", activation="relu"),
        layers.MaxPooling2D(pool_size=(3, 3), strides=2),
        layers.Flatten(),
        layers.Dropout(0.2),
        layers.Dense(2048, activation="relu"),
        layers.Dropout(0.2),
        layers.Dense(2048, activation="relu"),
        layers.Dense(num_classes, activation="softmax"),
    ]
    ),
    entropy=True
)

In [ ]:
epochs = 100
semi_model_1.compile(optimizer=keras.optimizers.Adam(), metrics=["accuracy"])

semi_model_1.fit(train_dataset, epochs=epochs)

Epoch 1/100
6/6 [==============================] - 10s 1s/step - loss: 0.0025 - accuracy: 0.5583
Epoch 2/100
6/6 [==============================] - 9s 2s/step - loss: 0.2414 - accuracy: 0.5917
Epoch 3/100
6/6 [==============================] - 9s 1s/step - loss: 0.0019 - accuracy: 0.6000
Epoch 4/100
6/6 [==============================] - 11s 2s/step - loss: 0.0594 - accuracy: 0.5667
Epoch 5/100
6/6 [==============================] - 5s 876ms/step - loss: 5.0570e-04 - accuracy: 0.5750
Epoch 6/100
6/6 [==============================] - 6s 931ms/step - loss: 0.1603 - accuracy: 0.5333
Epoch 7/100
6/6 [==============================] - 6s 973ms/step - loss: 0.0019 - accuracy: 0.6000
Epoch 8/100
6/6 [==============================] - 9s 2s/step - loss: 0.0475 - accuracy: 0.5750
Epoch 9/100
6/6 [==============================] - 6s 969ms/step - loss: 0.0195 - accuracy: 0.5750
Epoch 10/100
6/6 [==============================] - 5s 874ms/step - loss: 0.0018 - accuracy: 0.5833
Epoch 11/100
6/6 [

Model Saving
=================

In [ ]:
tf.saved_model.save(semi_model_1, "/content/drive/MyDrive/CSE164/FPModel/semi_model_1")
semi_model_1.save_weights('/content/drive/MyDrive/CSE164/FPModel/weights')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/CSE164/FPModel/semi_model_1/assets




---



###When doing the testing, you only need to run the set up part, make sure the weight is downloaded before starting here.

Load Model and Test Image
=================

In [ ]:
model = SemisupervisedModel(
    keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), padding="same", activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), padding="same", activation="relu"),
        layers.Conv2D(64, kernel_size=(3, 3), padding="same", activation="relu"),
        layers.Conv2D(64, kernel_size=(3, 3), padding="same", activation="relu"),
        layers.MaxPooling2D(pool_size=(3, 3), strides=2),
        layers.Flatten(),
        layers.Dropout(0.2),
        layers.Dense(2048, activation="relu"),
        layers.Dropout(0.2),
        layers.Dense(2048, activation="relu"),
        layers.Dense(num_classes, activation="softmax"),
    ]
    ),
    entropy=True
)
model.load_weights('/content/drive/MyDrive/CSE164/FPModel/weights')

In [ ]:
! unzip -o /content/ucsc-cse-164-spring-2022-final-project.zip "*/test/*" -d /content/MedMNIST

Streaming output truncated to the last 5000 lines.
  inflating: /content/MedMNIST/MedMNIST/test/64tdq2v5acpr0b3kl1js.jpeg  
  inflating: /content/MedMNIST/MedMNIST/test/64wpn30aymubhxsteg9z.jpeg  
  inflating: /content/MedMNIST/MedMNIST/test/64xwgi53tud7anfqj90k.jpeg  
  inflating: /content/MedMNIST/MedMNIST/test/64yua0nidshfzgvlxjcm.jpeg  
  inflating: /content/MedMNIST/MedMNIST/test/6512t4l3nj0we7f8vaqr.jpeg  
  inflating: /content/MedMNIST/MedMNIST/test/653uadwxvtenp1h0zlmf.jpeg  
  inflating: /content/MedMNIST/MedMNIST/test/65eg1p3yczvwskhb9408.jpeg  
  inflating: /content/MedMNIST/MedMNIST/test/65pzw8c0nqmirg3eyfdh.jpeg  
  inflating: /content/MedMNIST/MedMNIST/test/65w4ea17pvftcno2dhls.jpeg  
  inflating: /content/MedMNIST/MedMNIST/test/65wbfl3ok4172dntmxqg.jpeg  
  inflating: /content/MedMNIST/MedMNIST/test/675e8mr0jupznty9bfso.jpeg  
  inflating: /content/MedMNIST/MedMNIST/test/67erfmkvbjc5gdx8qyw0.jpeg  
  inflating: /content/MedMNIST/MedMNIST/test/67ju2gzhlm83s01vkfan.jpeg  


Prediction Generation
=================

In [ ]:
# load test data
test_images = []
filenames = os.listdir(PATH+'/test')
for i in range(len(filenames)):
    image_temp = tf.io.read_file(PATH+'/test/'+filenames[i])
    image_temp = tf.image.decode_jpeg(image_temp)
    test_images.append(image_temp)
test_images = np.array(test_images)
print("test data:", test_images.shape)

x_test = test_images.astype("float32") / 255

test data: (6000, 64, 64, 1)


In [ ]:
y_test = model.predict(x_test)
test_label = []
test_image_name = []
for i in range(len(filenames)):
  v = y_test[i].tolist()
  test_image_name.append(filenames[i][:-5])
  test_label.append(v.index(max(v)))

In [ ]:
a = np.asarray([test_image_name, test_label]).T
pd.DataFrame(a).to_csv('/content/drive/MyDrive/CSE164/predictions.csv', index=False, header=['image_id', 'label'])